In [1]:
import pandas as pd
import numpy as np
import math as mt
import datetime
import xlrd

#### Daily Return Metrics

In [3]:
#This cell reads both excel files
xls = pd.ExcelFile("2008_2009_constituent_aws_ranked.xlsx")
xls1 = pd.ExcelFile("LBO profile - 19y.xlsm")
positions = pd.read_excel(xls)
returns = pd.read_excel(xls1, sheets = 'Dailies')
positions.drop(0,axis = 0 ,inplace = True)
positions = positions.reset_index()
positions.drop(columns = ['index'], axis = 1, inplace = True)

In [4]:
usr = input("Please choose of the following numbers:\n \
            1. Verify the LBO100 portfolio return for a single day based on positions in the 2y hist from 2008-09.\n \
            2. Verify the LBO100 portfolio return for a range of dates based on positions in the 2y hist from 2008-09.\n")
avg = []
ret = []
final = pd.DataFrame(columns = ['Date', 'Calculated Returns','QMIT Reported Returns'])
if(usr == '1'):
    dt = input('Please enter the date in an mm/dd/yyyy format.\n')
    date = pd.to_datetime(dt, format="%m/%d/%Y")
    alist = positions.loc[positions['Periods'] == date]['Returns'].values
    blist = returns.loc[returns['Date'] == date]['LBO (T+1) Return'].values
    if alist.any():
        avg = np.average(alist)
        print("Calculated Portfolio Returns as per 2008_2009_constituent_aws_ranked.xlsx for requested date:", round(avg,2))
    else:
        print("Date not found in the Position dataset.")
    if blist.any():    
        ret = blist[0]
        print("QMIT Reported Portfolio Returns as per LBO profile - 19y.xlsm for requested date:", round(ret,2))
    else:
        print("Date not found in the Return dataset.")
elif(usr == '2'):
    dt = input('Please enter the start date in an mm/dd/yyyy format within 2008-2009.\n')
    dt1 = input('Please enter the end date in an mm/dd/yyyy format within 2008-2009.\n')
    date = pd.to_datetime(dt, format="%m/%d/%Y")
    date1 = pd.to_datetime(dt1, format="%m/%d/%Y")
    if(date>date1):
        print("Invalid Dates: End date is before the Start date. Please re-run the cell.")
    else:
        alist = positions.loc[(positions['Periods']>=date) & (positions['Periods']<=date1)]
        blist = returns.loc[(returns['Date']>=date) & (returns['Date']<=date1)]
        if alist['Returns'].values.any():
            for i in range(alist['Periods'].nunique()):
                avg.append(np.average(positions.loc[positions['Periods'] == alist['Periods'].unique()[i]]['Returns'].values))
            final.loc[:,'Date'] = pd.to_datetime(blist['Date'].unique())
            final.loc[:,'Calculated Returns'] = np.round(avg,2)
        else:
            print("Date not found in the Position dataset.")
        if blist['LBO (T+1) Return'].values.any():
            for i in range(blist['Date'].nunique()):
                ret.append(returns.loc[returns['Date'] == blist['Date'].unique()[i]]['LBO (T+1) Return'].values)
            final.loc[:,'QMIT Reported Returns'] = np.round(ret,2)
            print(final.to_string())
        else:
            print("Date not found in the Return dataset.")
else:
    print('The number chosen is invalid. Please re-run the cell and select either 1 or 2.')    

Please choose of the following numbers:
             1. Verify the LBO100 portfolio return for a single day based on positions in the 2y hist from 2008-09.
             2. Verify the LBO100 portfolio return for a range of dates based on positions in the 2y hist from 2008-09.
 2
Please enter the start date in an mm/dd/yyyy format within 2008-2009.
 01/01/2008
Please enter the end date in an mm/dd/yyyy format within 2008-2009.
 01/01/2010


          Date  Calculated Returns  QMIT Reported Returns
0   2008-01-02               -2.26                  -2.26
1   2008-01-03               -3.75                  -3.75
2   2008-01-04                0.71                   0.71
3   2008-01-07               -2.77                  -2.77
4   2008-01-08                0.43                   0.43
5   2008-01-09                0.82                   0.82
6   2008-01-10               -2.59                  -2.59
7   2008-01-11                1.74                   1.74
8   2008-01-14               -2.39                  -2.39
9   2008-01-15                1.74                   1.74
10  2008-01-16               -2.26                  -2.26
11  2008-01-17               -0.64                  -0.64
12  2008-01-18                0.42                   0.42
13  2008-01-22                3.63                   3.63
14  2008-01-23                0.53                   0.53
15  2008-01-24               -0.99                  -0.99
16  2008-01-25

#### Performance Metrics

In [5]:
#Replicating the Annualized Return for LBO over entire 19y
temp = []
for i in range(len(returns)):
    temp.append((1+returns['LBO (T+1) Return'].iloc[i]/100))
ret = (np.prod(temp)**(252/len(returns))-1)*100
print("Annualized LTD Return:", round(ret,2))

Annualized LTD Return: 22.21


In [6]:
#Replicating the Annualized Volatility & Downside Deviation
vol = pd.DataFrame(columns  = ['Value'])
# for i in range (len(returns)):
vol.loc['Vol_B', 'Value'] = returns['LBO (T+1) Return'].std()
vol.loc['Annualized Vol_B','Value'] = vol.loc['Vol_B','Value']*mt.sqrt(252)
vol.loc['Downside Dev','Value'] = np.sqrt(np.sum(returns[returns.loc[:,'LBO (T+1) Return']<0].loc[:,'LBO (T+1) Return'].values**2)/len(returns))
vol.loc['Ann Downside Dev','Value'] = vol.loc['Downside Dev','Value']*np.sqrt(252)
print("Annualized LTD Vol:", round(vol.loc['Annualized Vol_B','Value'],2))

Annualized LTD Vol: 23.82


In [7]:
#Sharpe
sharpe = ret/vol.loc['Annualized Vol_B','Value']
print("Sharpe:", round(sharpe,2))

Sharpe: 0.93


In [8]:
#Sortino
sortino = ret/vol.loc['Ann Downside Dev','Value']
print("Sortino:", round(sortino,2))

Sortino: 1.36


In [9]:
#Beta to S&P500
betasp = np.cov(returns['LBO (T+1) Return'], returns['SPX (T+1) Return'])[0][1]/np.cov(returns['LBO (T+1) Return'], returns['SPX (T+1) Return'])[1][1]
print("Beta to S&P500:", round(betasp,2))

Beta to S&P500: 1.09


In [10]:
#Skew & Kurtosis
temps = []
tempk = []
avg = np.average(returns['LBO (T+1) Return'])
std = np.std(returns['LBO (T+1) Return'])
for i in range(len(returns)):
    temps.append((returns['LBO (T+1) Return'].values[i] - avg)**3)  
    tempk.append((returns['LBO (T+1) Return'].values[i] - avg)**4)    
skew = (np.average(temps)/std**3)
kurt = (np.average(tempk)/std**4) - 3
print("Skew: ", round(skew,2))
print("Kurtosis: ", round(kurt,2))

Skew:  -0.09
Kurtosis:  4.56
